In [271]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import nltk, re, spacy, scipy
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack


from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.internals import find_jars_within_path

from nltk.tag import StanfordPOSTagger

from nltk.tag import StanfordNERTagger

from sklearn import linear_model

from sklearn.metrics import fbeta_score, accuracy_score



[nltk_data] Downloading package stopwords to /home/feujio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/feujio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
dataset = pd.read_csv("./dataset/medDataset2.csv")
dataset.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        1411 non-null   object
 1   url           1411 non-null   object
 2   Answer        619 non-null    object
 3   qtype         1411 non-null   object
 4   Focus         1411 non-null   object
 5   Question      1411 non-null   object
 6   Qtype-Coarse  1411 non-null   object
dtypes: object(7)
memory usage: 77.3+ KB


In [5]:
dataset

,source,url,Answer,qtype,Focus,Question,Qtype-Coarse
0,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Espaol\n \nInsomnia (in-SOM-ne-...,information,Insomnia,What is (are) Insomnia ?,Disease
1,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Secondary Insomnia\n \nSecondar...,causes,Insomnia,What causes Insomnia ?,Disease
2,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Insomnia is a common disorder. It affects wome...,susceptibility,Insomnia,Who is at risk for Insomnia? ?,Disease
3,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,The main symptom of insomnia is trouble fallin...,symptoms,Insomnia,What are the symptoms of Insomnia ?,Disease
4,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Your doctor will likely diagnose insomnia base...,exams and tests,Insomnia,How to diagnose Insomnia ?,Tests
...,...,...,...,...,...,...,...
1406,NINDS,http://www.ninds.nih.gov/disorders/neurosyphil...,The National Institute of Neurological Disorde...,research,Neurosyphilis,what research (or clinical trials) is being do...,Disease
1407,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Myoclonus refers to a sudden, involuntary jerk...",information,Myoclonus,What is (are) Myoclonus ?,Disease
1408,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,Treatment of myoclonus focuses on medications ...,treatment,Myoclonus,What are the treatments for Myoclonus ?,Drugs
1409,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Simple forms of myoclonus occur in normal, hea...",outlook,Myoclonus,What is the outlook for Myoclonus ?,Disease


In [6]:
dataset = dataset.convert_dtypes()  

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        1411 non-null   string
 1   url           1411 non-null   string
 2   Answer        619 non-null    string
 3   qtype         1411 non-null   string
 4   Focus         1411 non-null   string
 5   Question      1411 non-null   string
 6   Qtype-Coarse  1411 non-null   string
dtypes: string(7)
memory usage: 77.3 KB


In [9]:
qc_dataset = dataset[["Question", "qtype", "Qtype-Coarse"]]
qc_dataset = qc_dataset.rename(columns={"qtype":"Qtype-Fine"})

qc_dataset

,Question,Qtype-Fine,Qtype-Coarse
0,What is (are) Insomnia ?,information,Disease
1,What causes Insomnia ?,causes,Disease
2,Who is at risk for Insomnia? ?,susceptibility,Disease
3,What are the symptoms of Insomnia ?,symptoms,Disease
4,How to diagnose Insomnia ?,exams and tests,Tests
...,...,...,...
1406,what research (or clinical trials) is being do...,research,Disease
1407,What is (are) Myoclonus ?,information,Disease
1408,What are the treatments for Myoclonus ?,treatment,Drugs
1409,What is the outlook for Myoclonus ?,outlook,Disease


In [10]:
qc_dataset.describe()

,Question,Qtype-Fine,Qtype-Coarse
count,1411,1411,1411
unique,1387,16,4
top,What is the outlook for Electrocardiogram ?,information,Drugs
freq,3,198,950


In [174]:
train, test = train_test_split(qc_dataset, test_size=0.33, random_state=42, shuffle=True)

le2 = LabelEncoder()
le2.fit(pd.Series(train['Qtype-Coarse'].tolist() + test['Qtype-Coarse'].tolist()).values)
train['Qtype-Coarse'] = le2.transform(train['Qtype-Coarse'].values)
test['Qtype-Coarse'] = le2.transform(test['Qtype-Coarse'].values)
le3 = LabelEncoder()
le3.fit(pd.Series(train['Qtype-Fine'].tolist() + test['Qtype-Fine'].tolist()).values)
train['Qtype-Fine'] = le3.transform(train['Qtype-Fine'].values)
test['Qtype-Fine'] = le3.transform(test['Qtype-Fine'].values)




/home/feujio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/feujio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/feujio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

10

In [ ]:
######################################### 

In [290]:
kf = KFold(n_splits=10) # Define the split - into 2 folds 
kf.get_n_splits(train) # returns the number of splitting iterations in the cross-validator #print(KFold(n_splits=10, random_state=None, shuffle=False) )



KFold(n_splits=10, random_state=None, shuffle=False)

In [293]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)




NameError: name 'get_n_splits' is not defined

In [18]:
train.head()

,Question,Qtype-Fine,Qtype-Coarse
1363,What is Blond psyllium ?,5,1
1246,Are there safety concerns or special precautio...,10,1
1197,What is the action of American ginseng and how...,3,1
771,What is Biotin ?,5,1
631,Are there interactions between Papaya and othe...,8,1


In [164]:
all_corpus = pd.Series(train.Question.tolist() + test.Question.tolist()).astype(str)

In [165]:
def text_clean(corpus, keep_list):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)
    
    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process
    
    Output : Returns the cleaned text corpus
    
    '''
    cleaned_corpus = pd.DataFrame()
    for row in corpus:
        qs = []
        for word in row.split():
            if word not in keep_list:
                p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
                p1 = p1.lower()
                qs.append(p1)
            else : qs.append(word)
        aux = " ".join(qs)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)), ignore_index=True) #pd.DataFrame.from_records([{"Words" : aux}], index="Words"))
    return cleaned_corpus

In [166]:
res = text_clean(all_corpus, common_dot_words)

In [167]:
res.columns = ["questions"]

res

,questions
0,what is blond psyllium
1,are there safety concerns or special precautio...
2,what is the action of american ginseng and how...
3,what is biotin
4,are there interactions between papaya and othe...
...,...
1406,what is the action of folic acid and how does ...
1407,what is l arginine
1408,what are the symptoms of deep vein thrombosis
1409,what is are congenital heart defects


In [168]:
def preprocess(corpus, keep_list, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)
    
    Input : 
    'corpus' - Text corpus on which pre-processing tasks will be performed
    'keep_list' - List of words to be retained during cleaning process
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should 
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer
    
    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together
    
    Output : Returns the processed text corpus
    
    '''
    if cleaning == True:
        corpus = text_clean(corpus, keep_list) 
    
    corpus.columns = ["questions"]
    
    if remove_stopwords == True:
        wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
        stop = set(stopwords.words('english'))
        print()
        for word in wh_words:
            stop.remove(word)
        corpus = [[x for x in x.split() if x not in stop] for x in corpus["questions"]]
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        lem = WordNetLemmatizer()
        corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    
    if stemming == True:
        if stem_type == 'snowball':
            stemmer = SnowballStemmer(language = 'english') ## Comprendre
            corpus = [[stemmer.stem(x) for x in x] for x in corpus]
        else :
            stemmer = PorterStemmer()  ## Comprendre
            corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    
    corpus = [' '.join(x) for x in corpus]
        

    return corpus

In [169]:
common_dot_words = ['U.S.', 'St.', 'Mr.', 'Mrs.', 'D.C.']
all_corpus = preprocess(all_corpus, keep_list= common_dot_words, remove_stopwords = True)

In [170]:
all_corpus

['what blond psyllium',
 'safety concerns special precautions propolis',
 'what action american ginseng how work',
 'what biotin',
 'interactions papaya medications',
 'interactions grapefruit medications',
 'what action goji how work',
 'what action glucosamine sulfate how work',
 'what action eucalyptus how work',
 'what symptoms hemophilia',
 'what sarcoidosis',
 'interactions hops medications',
 'what treatments coronary microvascular disease',
 'what treatments cardiomyopathy',
 'what childhood interstitial lung disease',
 'how diagnose arrhythmia',
 'interactions wild yam foods',
 'what symptoms von willebrand disease',
 'who risk alpha 1 antitrypsin deficiency',
 'safety concerns special precautions glucosamine hydrochloride',
 'what sleep deprivation deficiency',
 'interactions reishi mushroom foods',
 'what causes bronchitis',
 'interactions niacin niacinamide vitamin b3 foods',
 'interactions phosphate salts medications',
 'what metabolic syndrome',
 'interactions gymnema foo

# Splitting the preprocessed combined corpus again into train and test set

In [175]:
train_corpus = all_corpus[0:train.shape[0]]
test_corpus = all_corpus[train.shape[0]:]

In [176]:
train_corpus

['what blond psyllium',
 'safety concerns special precautions propolis',
 'what action american ginseng how work',
 'what biotin',
 'interactions papaya medications',
 'interactions grapefruit medications',
 'what action goji how work',
 'what action glucosamine sulfate how work',
 'what action eucalyptus how work',
 'what symptoms hemophilia',
 'what sarcoidosis',
 'interactions hops medications',
 'what treatments coronary microvascular disease',
 'what treatments cardiomyopathy',
 'what childhood interstitial lung disease',
 'how diagnose arrhythmia',
 'interactions wild yam foods',
 'what symptoms von willebrand disease',
 'who risk alpha 1 antitrypsin deficiency',
 'safety concerns special precautions glucosamine hydrochloride',
 'what sleep deprivation deficiency',
 'interactions reishi mushroom foods',
 'what causes bronchitis',
 'interactions niacin niacinamide vitamin b3 foods',
 'interactions phosphate salts medications',
 'what metabolic syndrome',
 'interactions gymnema foo

In [180]:
nlp = spacy.load('en')

# Obtaining Features from Train Data, which would be fed to CountVectorizer

Creating list of Named Entitites, Lemmas, POS Tags, Syntactic Dependency Relation and Orthographic Features using shape.<br>
Later, these would be used as features for our model.

In [ ]:
# CountVectorizer is What? Is it a model?

In [196]:
all_ner = []
all_lemma = []
all_tag = []
all_dep = []
all_shape = []
all_pos_tag = []
for row in train_corpus:
    doc = nlp(row)
    present_lemma = []
    present_tag = []
    present_dep = []
    present_shape = []
    present_ner = []
    present_pos_tag = []
    #print(row)
    for token in doc:
        present_lemma.append(token.lemma_)
        present_pos_tag.append(token.pos_)
        present_tag.append(token.tag_)
        #print(present_tag)
        present_dep.append(token.dep_)
        present_shape.append(token.shape_)
    all_lemma.append(" ".join(present_lemma))
    all_pos_tag.append(" ".join(present_pos_tag))
    all_tag.append(" ".join(present_tag))
    all_dep.append(" ".join(present_dep))
    all_shape.append(" ".join(present_shape))
    for ent in doc.ents:
        present_ner.append(ent.label_)
    all_ner.append(" ".join(present_ner))

In [197]:
set(all_ner)

{'',
 'CARDINAL',
 'CARDINAL ORG',
 'GPE',
 'NORP',
 'NORP PERSON',
 'ORG',
 'ORG NORP',
 'ORG ORG',
 'PERSON',
 'TIME'}

In [203]:
all_pos_tag

['DET ADJ NOUN',
 'NOUN VERB ADJ NOUN NOUN',
 'DET NOUN PROPN PROPN ADV NOUN',
 'PRON PROPN',
 'PROPN PROPN NOUN',
 'NOUN PROPN NOUN',
 'DET NOUN VERB ADV NOUN',
 'DET NOUN NOUN NOUN ADV NOUN',
 'DET NOUN NOUN ADV NOUN',
 'PRON VERB PROPN',
 'DET NOUN',
 'NOUN VERB NOUN',
 'PRON VERB ADJ ADJ NOUN',
 'PRON VERB PROPN',
 'DET PROPN ADJ PROPN NOUN',
 'ADV VERB NOUN',
 'NOUN ADJ PROPN NOUN',
 'PRON PROPN PROPN PROPN PROPN',
 'PRON VERB NOUN NUM NOUN NOUN',
 'NOUN VERB ADJ NOUN NOUN NOUN',
 'DET VERB NOUN NOUN',
 'PROPN PROPN NOUN NOUN',
 'PRON VERB PROPN',
 'NOUN PROPN ADJ NOUN PROPN NOUN',
 'NOUN NOUN NOUN NOUN',
 'DET ADJ NOUN',
 'PROPN PROPN NOUN',
 'DET NOUN NOUN',
 'PROPN PROPN PROPN NOUN',
 'NOUN PROPN NOUN',
 'ADV ADJ NOUN',
 'PRON VERB NOUN NOUN',
 'PRON VERB ADJ NOUN NOUN',
 'PRON VERB NOUN NOUN',
 'ADV VERB ADJ ADJ NOUN',
 'PRON VERB ADJ NOUN',
 'PRON VERB NOUN PROPN',
 'DET NOUN PROPN NOUN',
 'DET NOUN NOUN',
 'PRON VERB NOUN NOUN',
 'DET NOUN ADJ ADJ NOUN',
 'PRON PROPN NOUN',


In [206]:
# all_lemma
all_tag
# spacy.explain("VBZ")

['WDT JJ NN',
 'NN VBZ JJ NNS NNS',
 'WDT NN NNP NNP WRB NN',
 'WP NNP',
 'NNPS NNP NNS',
 'NNS NNP NNS',
 'WDT NN VBD WRB NN',
 'WDT NN NN NN WRB NN',
 'WDT NN NN WRB NN',
 'WP VBZ NNP',
 'WDT NN',
 'NNS VBZ NNS',
 'WP VBZ JJ JJ NN',
 'WP VBZ NNP',
 'WDT NNP JJ NNP NN',
 'WRB VB NN',
 'NNS JJ NNP NNS',
 'WP NNPS NNP NNP NNP',
 'WP VBP NN CD NN NN',
 'NN VBZ JJ NNS NN NN',
 'WDT VB NN NN',
 'NNPS NNP NN NNS',
 'WP VBZ NNP',
 'NNS NNP JJ NN NNP NNS',
 'NNS NN NNS NNS',
 'WDT JJ NN',
 'NNPS NNP NNS',
 'WDT NN NN',
 'NNPS NNP NNP NNS',
 'NNS NNP NNS',
 'WRB JJ NN',
 'WP VBZ NN NN',
 'WP VBP JJ NN NN',
 'WP VBZ NN NN',
 'WRB VBP JJ JJ NN',
 'WP VBZ JJ NN',
 'WP VBZ NN NNP',
 'WDT NN NNP NN',
 'WDT NN NN',
 'WP VBZ NN NN',
 'WDT NN JJ JJ NN',
 'WP NNP NN',
 'NN VBZ JJ NNS JJ NN',
 'WP VBZ JJ JJ NN',
 'WRB JJ NN',
 'WP VBP JJ NN',
 'WP VBZ JJ NN',
 'NNS NNP NNS VBZ',
 'WP VBP NN NNS',
 'WP VBP NNP',
 'WP VBZ JJ NN NN',
 'NNS NNP NNPS',
 'WDT NN',
 'WRB JJ NN NN',
 'NN VBZ JJ NNS NNP',
 'WDT 

In [207]:
all_dep

['nsubj ROOT dobj',
 'nsubj compound amod compound ROOT',
 'det compound compound attr advmod ROOT',
 'det ROOT',
 'compound compound ROOT',
 'compound compound ROOT',
 'det nsubj ROOT advmod dobj',
 'dobj compound nsubj attr advmod ROOT',
 'det nsubj ROOT advmod ccomp',
 'det ROOT dobj',
 'det ROOT',
 'nsubj ROOT dobj',
 'nsubj ROOT amod amod dobj',
 'nsubj nsubj ROOT',
 'det compound amod compound ROOT',
 'advmod amod ROOT',
 'compound amod compound ROOT',
 'det compound compound compound ROOT',
 'nsubj ROOT compound nummod compound dobj',
 'compound nmod amod nsubj compound ROOT',
 'nsubj dobj compound ROOT',
 'compound compound compound ROOT',
 'nsubj ROOT dobj',
 'compound ROOT npadvmod compound compound ROOT',
 'nsubj ROOT compound dobj',
 'det compound ROOT',
 'compound compound ROOT',
 'det compound ROOT',
 'compound compound compound ROOT',
 'nsubj compound ROOT',
 'advmod amod ROOT',
 'nsubj ROOT compound dobj',
 'nsubj ROOT amod compound dobj',
 'nsubj ROOT compound dobj',
 

In [208]:
all_shape

['xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxx xxxx',
 'xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxx xxxx',
 'xxxx xxxx xxxx xxxx xxx xxxx',
 'xxxx xxxx xxxx xxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxxx',
 'xxx xxxx xxxx',
 'xxxx xxxx xxx xxxx',
 'xxxx xxxx xxx xxxx xxxx',
 'xxx xxxx xxxx d xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xd xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx x xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx xxxx xxxx',
 'xxxx xxxx xxxx',
 'xxxx xxxx 

In [212]:
count_vec_ner = CountVectorizer(ngram_range=(1, 2)).fit(all_ner)
ner_ft = count_vec_ner.transform(all_ner)

count_vec_lemma = CountVectorizer(ngram_range=(1, 2)).fit(all_lemma)
lemma_ft = count_vec_lemma.transform(all_lemma)

count_vec_tag = CountVectorizer(ngram_range=(1, 2)).fit(all_tag)
tag_ft = count_vec_tag.transform(all_tag)

count_vec_dep = CountVectorizer(ngram_range=(1, 2)).fit(all_dep)
dep_ft = count_vec_dep.transform(all_dep)

count_vec_shape = CountVectorizer(ngram_range=(1, 2)).fit(all_shape)
shape_ft = count_vec_shape.transform(all_shape)


In [228]:

# x_all_ft_train = hstack([ner_ft, lemma_ft, tag_ft, dep_ft, shape_ft]) #On garde celui-ci au frais! Trop de features. Malediction de la dimensionalite.
# x_all_ft_train

<945x1830 sparse matrix of type '<class 'numpy.longlong'>'
	with 21217 stored elements in COOrdinate format>

In [229]:
x_all_ft_train = hstack([ner_ft, lemma_ft, tag_ft])
x_all_ft_train

<945x1701 sparse matrix of type '<class 'numpy.longlong'>'
	with 12501 stored elements in COOrdinate format>

# Now we will obtain the Feature vectors for the test set using the CountVectorizers Obtained from the Training Corpus


In [230]:
all_test_ner = []
all_test_lemma = []
all_test_tag = []
all_test_dep = []
all_test_shape = []
for row in test_corpus:
    doc = nlp(row)
    present_lemma = []
    present_tag = []
    present_dep = []
    present_shape = []
    present_ner = []
    #print(row)
    for token in doc:
        present_lemma.append(token.lemma_)
        present_tag.append(token.tag_)
        #print(present_tag)
        present_dep.append(token.dep_)
        present_shape.append(token.shape_)
    all_test_lemma.append(" ".join(present_lemma))
    all_test_tag.append(" ".join(present_tag))
    all_test_dep.append(" ".join(present_dep))
    all_test_shape.append(" ".join(present_shape))
    for ent in doc.ents:
        present_ner.append(ent.label_)
    all_test_ner.append(" ".join(present_ner))

In [231]:
ner_test_ft = count_vec_ner.transform(all_test_ner)
lemma_test_ft = count_vec_lemma.transform(all_test_lemma)
tag_test_ft = count_vec_tag.transform(all_test_tag)
dep_test_ft = count_vec_dep.transform(all_test_dep)
shape_test_ft = count_vec_shape.transform(all_test_shape)

In [233]:
#x_all_ft_test = hstack([ner_test_ft, lemma_test_ft, tag_test_ft, dep_test_ft, shape_test_ft])
x_all_ft_test = hstack([ner_test_ft, lemma_test_ft, tag_test_ft])

In [236]:
type(x_all_ft_test)

scipy.sparse.csr.csr_matrix

In [235]:
x_all_ft_test = x_all_ft_test.tocsr()  #Pourquoi on compresse ces donnees de test
x_all_ft_test

<466x1701 sparse matrix of type '<class 'numpy.longlong'>'
	with 5808 stored elements in Compressed Sparse Row format>

# Model Training

In [265]:
model = svm.LinearSVC(verbose=True)   #Checks also SVC

In [260]:
# help(svm.SVC)

In [266]:
model.fit(x_all_ft_train, train['Qtype-Coarse'].values)

[LibLinear]

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=True)

# Model Evaluation

In [268]:
preds = model.predict(x_all_ft_test)

In [269]:
preds

array([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 2, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 1, 1, 1, 2, 1,
       0, 2, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0,
       2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,

In [272]:
accuracy_score(test['Qtype-Coarse'].values, preds)

0.9763948497854077

In [282]:
model.fit(x_all_ft_train, train['Qtype-Fine'].values)

[LibLinear]

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=True)

In [283]:
preds = model.predict(x_all_ft_test)

In [284]:
preds

array([14,  7,  7,  0, 10, 15,  0,  5,  8, 15, 15, 13,  7,  3,  5,  4,  6,
        0,  5,  8, 11,  1, 11, 14,  4,  4,  3,  1, 14, 10, 14,  3,  1,  4,
        2,  6,  4, 13,  2,  5,  1, 10,  1, 13,  8, 14, 14,  4,  7, 15,  1,
        7,  7,  5, 13,  8,  8,  5,  1,  3, 10, 15, 12,  1, 10, 14, 11,  5,
        5,  5,  8, 13,  5,  5, 11,  3,  0,  5,  4,  3,  0,  4,  0, 15,  2,
        8, 10,  6, 13,  5,  1,  3,  5, 14,  5,  4,  0, 11, 15,  0,  3, 13,
        5,  7,  2,  6,  8, 10,  2,  8,  0,  2,  0, 15, 14, 11,  0,  5,  5,
        6,  8,  4,  1,  0,  3, 10, 10, 12,  3, 15,  4,  5,  6,  1,  6, 14,
        5,  3,  8,  5, 10, 15,  1,  0,  2, 10,  0,  1, 10,  7,  4,  6,  3,
       14,  8,  3,  3, 11,  5,  5,  7,  2, 10, 15,  3, 15,  3,  8, 15,  4,
        0, 13,  9,  0,  1,  1, 10, 15,  3,  2,  4,  0,  5, 10,  1,  1, 10,
        8,  2, 10,  1,  7,  8,  1, 10,  6,  4, 15, 11,  0,  4,  5,  5,  2,
        4, 14,  4,  2,  3, 10,  7,  3,  5,  8,  3, 15, 15,  1, 15, 15, 14,
        0,  4, 14,  4,  4

In [288]:
accuracy_score(test['Qtype-Fine'].values, preds)

1.0

In [ ]:
# Train set, Validation, Test set